# Looking for the objects in the catalogs

We will try to find the onjects from the lst in the catalogs TeV, GeV and Simbad.

- PSR B1259-63
- LS 5039
- LS   I +61  132
- HESS J0632+057
- PSR J2032+4127
- 1FGL J1018.6-5856
- HESS J0536-675


In [3]:
import pandas as pd
import re
import sys

In [4]:
regular_expressions = ['.*B12.*63.*',
                       '.*LS.*5039.*',
                       '.*LS.*I.*61.*303.*',
                       '.*J0632.*057.*',
                       '.*J2032.*.*4127.*',
                       '.*1FGL.*J1018.*5856.*',
                       '.*J0536.*67.*',
                      ]

This cell loads data created in simbad.ipynb

In [7]:
cat_name = "gevtevsimbadxmm_005"
tevsimbad = pd.read_csv("data/tevsimbad.txt", sep='\t', index_col=0)
gevsimbad = pd.read_csv("data/gevsimbad.txt", sep='\t', index_col=0)
gevtevsimbad = pd.read_csv("data/gevtevsimbad.txt", sep='\t', index_col=0)
pd_gev = pd.read_csv("data/gev.txt", sep='\t', index_col=0)
pd_tev = pd.read_csv("data/tev.txt", sep='\t', index_col=0)
pd_simbad = pd.read_csv("data/simbad.txt", sep='\t', index_col=0)
pd_gevtev_simbadclasses = pd.read_csv("data/gevtev_simbadclasses.txt", sep='\t', index_col=0)
pd_gevtev_simbadclasses = pd_gevtev_simbadclasses.loc[:, ~pd_gevtev_simbadclasses.columns.str.contains('^Unnamed')]
pd_gevtev_simbadclasses = pd_gevtev_simbadclasses.reset_index(drop=True)
pd_gevtev_simbadclasses.to_csv("data/gevtev_simbadclasses.txt", sep='\t')
pd_gevtevxmm_simbadclasses = pd.read_csv("data/{}.txt".format(cat_name), sep = '\t', index_col=0)
pd_gevtevxmm_simbadclasses = pd_gevtevxmm_simbadclasses.loc[:, ~pd_gevtevxmm_simbadclasses.columns.str.contains('^Unnamed')]
pd_gevtevxmm_simbadclasses = pd_gevtevxmm_simbadclasses.reset_index(drop=True)
pd_gevtevxmm_simbadclasses.to_csv("data/{}.txt".format(cat_name), sep='\t')

In [8]:
print(len(pd_gevtev_simbadclasses))
print(len(pd_gevtevxmm_simbadclasses))

199
1343


The following cell will be useful to determine a string name of a catalog where an object is found. It is not the cleverest way to do it, as if number of elements is changed(another epsilon is used), this dictionary should be corrected manually.

In [9]:
len_catalogs = {len(tevsimbad) : 'tevsimbad',
            len(gevsimbad) :'gevsimbad',
            len(gevtevsimbad) :'gevtevsimbad',
            len(pd_gev) : 'gev',
            len(pd_tev) : 'tev',
            len(pd_simbad) : 'simbad',
            len(pd_gevtev_simbadclasses) : 'gevtev_simbadclasses',
            len(pd_gevtevxmm_simbadclasses) : cat_name
               }

In [10]:
name_to_catalog = {'tevsimbad' : tevsimbad,
            'gevsimbad' : gevsimbad,
            'gevtevsimbad' : gevtevsimbad,
            'gev' : pd_gev,
            'tev' : pd_tev,
            'simbad' : pd_simbad,
            'gevtev_simbadclasses' : pd_gevtev_simbadclasses,
            cat_name : pd_gevtevxmm_simbadclasses}

In [11]:
gev_columns = ['gev_2FGL_Name', 'gev_1FGL_Name', 
                 'gev_1FHL_Name', 'gev_2FGL_Name', 
                 'gev_ASSOC1', 'gev_ASSOC2',
                'gev_ASSOC_GAM1', 'gev_ASSOC_GAM2',
                'gev_ASSOC_GAM3', 'gev_ASSOC_TEV', 'gev_CLASS1',
                'gev_GLAT', 'gev_GLON']

tev_columns = ['tev_common_name', 'tev_gamma_names', 
                'tev_fermi_names', 'tev_other_names', 
                'tev_classes',
                'tev_glat', 'tev_glon']

s_columns = ['s_class', 's_spectra', 's_glat', 's_glon',
            's_MAIN_ID']

xmm_columns = ['xmm_IAUNAME']

s_tev_columns = tev_columns + s_columns
s_gev_columns = gev_columns + s_columns
tev_gev_columns = gev_columns + tev_columns
tev_gev_columns_s = gev_columns + tev_columns + ['s_class', 's_simbad', 's_MAIN_ID']
gevtevs_columns = gev_columns + tev_columns + s_columns
tev_gev_xmm_columns_s = gev_columns + tev_columns + xmm_columns + ['s_class', 's_simbad', 's_MAIN_ID']

In [12]:
name_to_column = {'tevsimbad' : s_tev_columns,
            'gevsimbad' : s_gev_columns,
            'gevtevsimbad' : gevtevs_columns,
            'gev' : gev_columns,
            'tev' : tev_columns,
            'simbad' : s_columns,
            'gevtev_simbadclasses' : tev_gev_columns_s,
            cat_name : tev_gev_xmm_columns_s}

In [13]:
def find_in_catalogs(string_to_find, catalog):
    """Looks for an object containing in its name string_to_find in the catalog
    
    Inputs:
    string_to_find - string containing a part og a name that we are looking for
    catalog - in which catalog to look for (pandas DataFrame)
    """
    for c in catalog.columns:
        for obj in catalog[c]:
            if (str(obj)).find(string_to_find) != -1:
                print(len_catalogs[len(catalog)] + ' :')
                print('\t' + str(c))
                print('\t\t' + (str(obj)))
            


In [14]:
def old_find_number_in_catalogs(string_to_find, catalog, printRes = False):
    #Based on string find
    """Looks for an object containing in its name string_to_find in the catalog,
    Finds its number in the corresponging catalog
    
    Inputs:
    string_to_find - string containing a part og a name that we are looking for
    catalog - in which catalog to look for (pandas DataFrame)
    printRes - True if you want to see the search
    
    Returns:
    success - if found is equal to 1, if not - 0
    catalog_name - name of the catalog where string was found(string)
    number_obj - number of a corresponding line
    
    """
    success = 0
    catalog_name = '' 
    number_obj = -1
    for c in catalog.columns:
        for number_obj in range(len(catalog[c])):

            obj = catalog[c][number_obj]
            if (str(obj)).find(string_to_find) != -1:
                success = 1
                
                catalog_name = len_catalogs[len(catalog)]
                
                if printRes:
                    print(len_catalogs[len(catalog)] + ' :')
                    print('\t' + str(c))
                    print('\t\t' + (str(obj)))
                    print('\t\t\t' + str(number_obj))
                return success, catalog_name, number_obj
                
    return success, catalog_name, number_obj

In [15]:
def find_number_in_catalogs(regular_expression, catalog, printRes = False):
    #based on regular expressions
    """Looks for an object containing in its name string_to_find in the catalog,
    Finds its number in the corresponging catalog
    
    Inputs:
    string_to_find - string containing a part og a name that we are looking for
    catalog - in which catalog to look for (pandas DataFrame)
    printRes - True if you want to see the search
    
    Returns:
    success - if found is equal to 1, if not - 0
    catalog_name - name of the catalog where string was found(string)
    number_obj - number of a corresponding line
    
    """
    success = 0
    catalog_name = '' 
    number_obj = -1
    for c in catalog.columns:
        for number_obj in range(len(catalog[c])):
            try:
                obj = catalog[c][number_obj]
            except:
                print(c)
                display(catalog[c])
                print(number_obj)
            if (re.search(regular_expression, str(obj)) is not None):
                success = 1
                
                catalog_name = len_catalogs[len(catalog)]
                
                if printRes:
                    print(len_catalogs[len(catalog)] + ' :')
                    print('\t' + str(c))
                    print('\t\t' + (str(obj)))
                    print('\t\t\t' + str(number_obj))
                return success, catalog_name, number_obj
                
    return success, catalog_name, number_obj

In [16]:
catalogs = [#gevtevsimbad,
            #tevsimbad,
            #gevsimbad,
            pd_gevtev_simbadclasses,
            pd_gevtevxmm_simbadclasses
           ]

In [17]:
separate_catalogs = [
    pd_gev,
    pd_tev,
    pd_simbad,
]

Strings we want to find in the catalogs:

In [18]:
strings_to_find = [
    'PSR B1259-63',
    'LS 5039',
    'LS   I +61  303',
    'HESS J0632+057',
    'LAT PSR J2032+4127',
    '1FGL J1018.6-5856',
    'J0536',
    
]

In [19]:
resultats_of_search = {}
#this dictionary remembers catalog name and corresponding number of line where an interesting objects is found
for i in len_catalogs.values():
    resultats_of_search.update({i : []})

In [20]:
for regular_expression in regular_expressions:
    print('Looking for : ' + regular_expression)
    for j in catalogs:

        success, catalog_name, number_obj = find_number_in_catalogs(regular_expression, j)
        print(len_catalogs[len(j)])
        if success:
            resultats_of_search[catalog_name].append(number_obj)
print('Done')

Looking for : .*B12.*63.*
gevtev_simbadclasses
gevtevsimbadxmm_005
Looking for : .*LS.*5039.*
gevtev_simbadclasses
gevtevsimbadxmm_005
Looking for : .*LS.*I.*61.*303.*
gevtev_simbadclasses
gevtevsimbadxmm_005
Looking for : .*J0632.*057.*
gevtev_simbadclasses
gevtevsimbadxmm_005
Looking for : .*J2032.*.*4127.*
gevtev_simbadclasses
gevtevsimbadxmm_005
Looking for : .*1FGL.*J1018.*5856.*
gevtev_simbadclasses
gevtevsimbadxmm_005
Looking for : .*J0536.*67.*
gevtev_simbadclasses
gevtevsimbadxmm_005
Done


In [21]:
print(resultats_of_search)

{'gevtevsimbadxmm_005': [1091, 160, 406], 'gevsimbad': [], 'tevsimbad': [], 'simbad': [], 'gevtevsimbad': [], 'tev': [], 'gev': [], 'gevtev_simbadclasses': [152, 13, 188, 48]}


Looking in separate catalogs:

In [22]:
#for regular_expression in regular_expressions:
#    print('Looking for : ' + regular_expression)
#    for j in separate_catalogs:
#        success, catalog_name, number_obj = find_number_in_catalogs(regular_expression, j)
#        if success:
#            resultats_of_search[catalog_name].append(number_obj)
#print('Done')

In [23]:
print(resultats_of_search)

{'gevtevsimbadxmm_005': [1091, 160, 406], 'gevsimbad': [], 'tevsimbad': [], 'simbad': [], 'gevtevsimbad': [], 'tev': [], 'gev': [], 'gevtev_simbadclasses': [152, 13, 188, 48]}


In [24]:
pd_gevtevxmm_simbadclasses 

,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,xmm_SC_HR3,xmm_SC_HR3_ERR,xmm_SC_HR4,xmm_SC_HR4_ERR,xmm_SC_POSERR,xmm_SC_RA,xmm_SC_SUM_FLAG,xmm_SC_VAR_FLAG,xmm_SRCID,xmm_WEBPAGE_URL
0,6496,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.619995,0.053219,-0.996104,0.035598,1.650980,6.264828,4,70,200962101010080,b'http://xmm-catalog.irap.omp.eu/source/200962...
1,6497,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.621372,0.043878,-1.000000,0.016441,1.550980,6.267377,3,70,203105901010059,b'http://xmm-catalog.irap.omp.eu/source/203105...
2,6498,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.964721,0.097340,-1.000000,0.135793,1.271590,6.267657,3,70,205111801010051,b'http://xmm-catalog.irap.omp.eu/source/205111...
3,6499,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.731011,0.017351,-0.999620,0.010612,0.642101,6.271308,3,84,200962101010052,b'http://xmm-catalog.irap.omp.eu/source/200962...
4,6501,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.639933,0.003315,-0.998643,0.001549,0.532384,6.271569,3,84,200962101010042,b'http://xmm-catalog.irap.omp.eu/source/200962...
5,6503,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.850549,0.081620,-0.906882,0.093912,1.653910,6.271974,3,70,205111801010059,b'http://xmm-catalog.irap.omp.eu/source/205111...
6,6504,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.787896,0.066574,-0.984689,0.053113,1.671570,6.273209,3,70,203105901010079,b'http://xmm-catalog.irap.omp.eu/source/203105...
7,6507,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.715384,0.003504,-0.999745,0.001211,0.534570,6.275046,4,84,200962101010041,b'http://xmm-catalog.irap.omp.eu/source/200962...
8,6508,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.647709,0.033139,-1.000000,0.019354,1.237100,6.277828,3,84,203105902010047,b'http://xmm-catalog.irap.omp.eu/source/203105...
9,6509,b' ',b' ',b' ',b' ',b'Tycho ',b' ',b' ',b' ',b' ',...,-0.690540,0.035031,-1.000000,0.019152,1.073310,6.278226,3,84,200962101010063,b'http://xmm-catalog.irap.omp.eu/source/200962...


In [26]:
pd_gevtevxmm_simbadclasses.iloc[resultats_of_search[cat_name]]

,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,xmm_SC_HR3,xmm_SC_HR3_ERR,xmm_SC_HR4,xmm_SC_HR4_ERR,xmm_SC_POSERR,xmm_SC_RA,xmm_SC_SUM_FLAG,xmm_SC_VAR_FLAG,xmm_SRCID,xmm_WEBPAGE_URL
1091,414319,b'0FGL J1826.3-1451',b'1FGL J1826.2-1450 ',b'1FHL J1826.2-1450 ',b'2FGL J1826.3-1450 ',b'LS 5039 ',b' ',b' ',b' ',b' ',...,0.353358,0.556636,-0.103077,0.44328,1.38185,282.222293,0,0,203061702010015,b'http://xmm-catalog.irap.omp.eu/source/203061...
160,69036,b'0FGL J0240.3+6113',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',...,-0.606868,0.331495,-1.000000,1.79392,2.16896,40.045206,0,0,205059814010048,b'http://xmm-catalog.irap.omp.eu/source/205059...
406,211987,b' ',b' ',b' ',b'2FGL J1016.5-5858 ',b'PSR J1016-5857 ',b' ',b' ',b'3EG J1013-5915',b' ',...,-0.363046,0.491091,-1.000000,1.58322,2.30205,154.499492,0,0,206943901010110,b'http://xmm-catalog.irap.omp.eu/source/206943...


In [27]:
for i in resultats_of_search.keys():
    if i.find('simbad') != -1 and i.find('simbadclasses') == -1:
        continue
    print(i + ' :')
    display(name_to_catalog[i][name_to_column[i]].loc[resultats_of_search[i]])

tev :


,tev_common_name,tev_gamma_names,tev_fermi_names,tev_other_names,tev_classes,tev_glat,tev_glon


gev :


,gev_2FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,gev_ASSOC_TEV,gev_CLASS1,gev_GLAT,gev_GLON


gevtev_simbadclasses :


,gev_2FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,gev_ASSOC_TEV,...,tev_common_name,tev_gamma_names,tev_fermi_names,tev_other_names,tev_classes,tev_glat,tev_glon,s_class,s_simbad,s_MAIN_ID
152,b'2FGL J1826.3-1450 ',b'1FGL J1826.2-1450 ',b'1FHL J1826.2-1450 ',b'2FGL J1826.3-1450 ',b'LS 5039 ',b' ',b' ',b' ',b' ',b'LS 5039 ',...,b'LS 5039',b'HESS J1826-148',b'',b'RX J1826.2-1450',b'bin',-1.289216,16.881586,['B0Ve'],1,"[""b'V* V479 Sct'""]"
13,b'2FGL J0240.5+6113 ',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',b'LS I +61 303 ',...,b'LS I +61 303',b'VER J0240+612',b'',b'',b'bin',1.086135,135.675278,['B0Ve'],1,"[""b'LS I +61 303'"" ""b'LS I +61 303'""]"
188,b'2FGL J2032.2+4126 ',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',b'TeV J2032+4130 ',...,b'MGRO J2031+41',b'',b'',"b'TeV J2032+4130,TeV 2032+413,0FGL J2032.2+4122'",b'unid',1.111117,80.299477,['B0Ve'],1,"[""b'[MT91] 213'"" ""b'Schulte 21'""]"
48,b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b'1FHL J1018.9-5855 ',b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b' ',b' ',b' ',b' ',b'HESS J1018-589 ',...,b'HESS J1018-589 B',b'HESS J1018-589 B',b'',b'PSR J1016-5857',b'pwn',-1.879876,284.079346,[],0,[]


The following cell saves information about objects found.

In [22]:
#gevtevsimbad[gevtevs_columns].to_csv('resultat_gevtevsimbad.txt',sep = '\t')
#gevtev[tev_gev_columns].to_csv('resultat_gevtev.txt',sep = '\t')
#gevsimbad[s_gev_columns].to_csv('resultat_sgev.txt',sep = '\t')
#tevsimbad[s_tev_columns].to_csv('resultat_stev.txt',sep = '\t')

In [23]:
#for i in resultats_of_search.keys():
#    (name_to_catalog[i].loc[resultats_of_search[i]]).to_csv('resultats/resultat_' + i +'.txt',sep = '\t')

The following cell adds to bases used in the beginning a column that indicates the objects choosen in this ipynb.

In [28]:
for i in resultats_of_search.keys():
    print(i)
    if i in ['gev','tev','simbad']:
        continue
    catalog = name_to_catalog[i].copy()
    catalog.is_copy = False
    catalog['marked'] = 0
    (catalog.loc[resultats_of_search[i], 'marked'])= 1
    (catalog).to_csv('markeddata/' + i +'.txt',sep = '\t')


gevtevsimbadxmm_005
gevsimbad
tevsimbad
simbad
gevtevsimbad
tev
gev
gevtev_simbadclasses
